# xpander.ai single agent using local tools for mathematics calculations combined with OpenAI

In [ ]:
from xpander_sdk import XpanderClient, ToolCallResult, GraphItem
from openai import OpenAI
from dotenv import load_dotenv
from os import environ

load_dotenv()


In [2]:
OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]

## Initialize xpander.ai Client

In [3]:
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)

## Initialize OpenAI Client

In [4]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

## Load agent

In [ ]:
agent = xpander_client.agents.create(name="Calculator Agent")
print(f"created agent {agent.id} - https://app.xpander.ai/agents/{agent.id}")

## Create local tools

In [6]:
# local tools declerations
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def square(a: int) -> int:
    """Multiply a by itself and returns the result integer"""
    return a*a

# store the local tools as an array
local_tools = [
    {
        "declaration": {
            "type": "function",
            "function": {
                "name": "multiply",
                "description": "Multiply two integers and returns the result integer",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {
                            "type": "integer",
                            "description": "First integer to multiply"
                        },
                        "b": {
                            "type": "integer",
                            "description": "Second integer to multiply"
                        }
                    },
                    "required": ["a", "b"]
                }
            }
        },
        "fn": multiply
    },
    {
        "declaration": {
            "type": "function",
            "function": {
                "name": "add",
                "description": "Add two integers and returns the result integer",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {
                            "type": "integer",
                            "description": "First integer to add"
                        },
                        "b": {
                            "type": "integer",
                            "description": "Second integer to add"
                        }
                    },
                    "required": ["a", "b"]
                }
            }
        },
        "fn": add
    },
    {
        "declaration": {
            "type": "function",
            "function": {
                "name": "square",
                "description": "Multiply a by itself and returns the result integer",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {
                            "type": "integer",
                            "description": "Integer to square"
                        }
                    },
                    "required": ["a"]
                }
            }
        },
        "fn": square
    }
]

local_tools_list = [tool['declaration'] for tool in local_tools] # helper
local_tools_by_name = {} # helper

for tool in local_tools:
    local_tools_by_name[tool['declaration']['function']['name']] = tool['fn']


## Attach the local tools to the agent

In [7]:
agent.add_local_tools(local_tools_list)

## Add the tools to the graph (Optional)

In [ ]:
# define the nodes on the graph
multiply_node = agent.graph.add_node(GraphItem(agent=agent,item_id="multiply", name="Multiply number", is_local_tool=True))
square_node = agent.graph.add_node(GraphItem(agent=agent,item_id="square", name="Square number", is_local_tool=True))
add_node = agent.graph.add_node(GraphItem(agent=agent,item_id="add", name="Add number", is_local_tool=True))

# connect edges (square -> square (repetitive) -> add -> add (repetitive))
square_node.connect([square_node,add_node])
add_node.connect([add_node])

## Sync (deploy)

In [ ]:
agent.sync()

## Invoke the agent with query and create an execution task

In [ ]:
agent.add_task("""
What is the square of 14 + the square of 12? Calculate step by step
""")

## Init the agent's memory with input (query) and instructions

In [9]:
agent.memory.init_messages(input=agent.execution.input_message,instructions=agent.instructions)

## Run the agent and wait for result

In [10]:
while not agent.is_finished():
    response = openai_client.chat.completions.create(
                model= "gpt-4o",
                messages=agent.messages,
                tools=agent.get_tools(),
                tool_choice=agent.tool_choice,
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    agent.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    # run tools
    agent.run_tools(tool_calls=tool_calls)
    
    # extract pending local tools executions
    pending_local_tool_execution = XpanderClient.retrieve_pending_local_tool_calls(tool_calls=tool_calls)
    local_tools_results = []
    
    # iterate local tools and run them
    for tc in pending_local_tool_execution:
        # create result
        tool_call_result = ToolCallResult(function_name=tc.name,tool_call_id=tc.tool_call_id,payload=tc.payload)
        try:
            if tc.name in local_tools_by_name:
                tool_call_result.is_success = True
                tool_call_result.result = local_tools_by_name[tc.name](**tc.payload)
            else:
                raise Exception(f"Local Tool {tc.name} not found!")
        except Exception as e:
            tool_call_result.is_success = False
            tool_call_result.is_error = True
            tool_call_result.result = str(e)
        finally:
            local_tools_results.append(tool_call_result)
    
    # report the execution result to the memory
    if len(local_tools_results) != 0:
        agent.memory.add_tool_call_results(tool_call_results=local_tools_results)

## Fetch & print the agent execution result

In [ ]:
execution_result = agent.retrieve_execution_result()

print("status", execution_result.status)
print("result", execution_result.result)